# Projet Speed Dating (Tinder)

Cheminement du nettoyage apporté au dataset avant de poursuivre l'analyse.

## Import de modules externes

In [ ]:
import requests
import pandas as pd

from speed_dating_document_context import (
    LabelDecoder,
    pref_1_1_cols,
    pref_4_1_cols,
    pref_2_1_cols,
    pref_1_2_cols
)

from config import (
    JEDHA_SPEED_DATING_DATA_EXPLANATIONS_URL,
    LOCAL_SPEED_DATING_DATA_EXPLANATIONS_PATH,
    JEDHA_SPEED_DATING_CSV_URL,
    LOCAL_SPEED_DATING_CSV_PATH,
)

# display settings
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', 1000)          # Expand display width
pd.set_option('display.max_colwidth', None)   # Show full content in each cell

## Chargement des données

In [12]:
# Retrieve locally explanation document
if not LOCAL_SPEED_DATING_DATA_EXPLANATIONS_PATH.exists():
    response = requests.get(JEDHA_SPEED_DATING_DATA_EXPLANATIONS_URL)

    with open(LOCAL_SPEED_DATING_DATA_EXPLANATIONS_PATH, mode="wb") as file:
        file.write(response.content)
        
if not LOCAL_SPEED_DATING_CSV_PATH.exists():
    response = requests.get(JEDHA_SPEED_DATING_CSV_URL)

    with open(LOCAL_SPEED_DATING_CSV_PATH, mode="wb") as file:
        file.write(response.content)

# Loading DataFrame
speed_dating_df = pd.read_csv(LOCAL_SPEED_DATING_CSV_PATH, encoding="ISO-8859-1")

## Simple data cleaning

### Null PID

In [13]:
waves_with_null_pid = speed_dating_df[speed_dating_df["pid"].isna()]["wave"].unique()
print("Waves where a pid is null: ", [pid.item() for pid in waves_with_null_pid])


Waves where a pid is null:  [5]


In [14]:
wave5_df = speed_dating_df[speed_dating_df["wave"] == 5]

# Retrieve list of participant without a partner
wave5_iids_with_null_pid = wave5_df[wave5_df["pid"].isna()]
iid_order_list = wave5_iids_with_null_pid[["iid", "order"]].values.tolist()

peer_dates = 0

for iid, order in iid_order_list:
    has_peer_date = len(wave5_df[(wave5_df["pid"] == iid) & (wave5_df["order"] == order)]) > 0
    if has_peer_date:
        print(f"FOUND peer date for {iid} / {order}")
        peer_dates = peer_dates + 1

print(f"Check done! Found {peer_dates} peer dates.")

Check done! Found 0 peer dates.


No peers found, we can simply remove those lines.
Maybe the expected participant decided not to replies to follow-up polls.

In [15]:
speed_dating_df.dropna(subset=["pid"], ignore_index=True, inplace=True)

speed_dating_df

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,1,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,2,12.0,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,3,13.0,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,4,14.0,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,1,7.0,8.0,9.0,8.0,9

### Normalisation des notes

Pour les préferences dons l'échelle de notation varie, on va adapter à celle du plus grand nombre: distribution de 100 points.
Pour les vagues 6 à 9 on normalisera les valeurs des groupes suivant:
- attr1_1, sinc1_1, intel1_1, fun1_1, amb1_1, shar1_1
- attr4_1, sinc4_1, intel4_1, fun4_1, amb4_1, shar4_1
- attr2_1, sinc2_1, int2_1, fun2_1, amb2_1, shar2_1
- attr1_2, sinc1_2, intel1_2, fun1_2, amb1_2, shar1_2

In [16]:
# enforce boundaries for abnormal values (could be a global sanitization applied on all columns expecting a range)
abnormal_cols = ["attr_o", "fun_o", "gaming", "reading"]
display(speed_dating_df[abnormal_cols].max())

speed_dating_df[abnormal_cols] = speed_dating_df[abnormal_cols].clip(lower=1, upper=10)

display(speed_dating_df[abnormal_cols].max())
display(speed_dating_df.describe(include='all'))


attr_o     10.5
fun_o      11.0
gaming     14.0
reading    13.0
dtype: float64

attr_o     10.0
fun_o      10.0
gaming     10.0
reading    10.0
dtype: float64

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
count,8368.000000,8367.000000,8368.00000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,6532.000000,8368.000000,8368.000000,8368.000000,8368.000000,8210.000000,8368.000000,8274.000000,8305.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,8368.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,8274.000000,8305,8286.000000,4914,3133,3583,8305.000000,8289.000000,8289.000000,8289,7305,4273,8289.000000,8271.000000,8289.000000,8279,8230.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8267.000000,1790.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,6489.000000,6489.000000,6489.000000,6489.000000,6489.000000,6467.000000,8289.000000,8289.000000,8289.000000,8289.000000,8280.000000,8280.000000,8263.000000,8263.000000,8263.000000,8263.000000,8263.000000,4906.000000,4906.000000,4906.000000,4906.000000,4906.000000,8368.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,7195.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,7454.000000,7454.000000,7424.000000,1984.000000,1955.000000,1984.000000,1984.000000,1955.000000,1974.000000,7436.000000,7454.000000,7454.000000,7454.000000,7454.000000,7454.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.00000,5775.000000,5775.000000,5775.000000,5775.000000,7454.000000,7454.000000,7454.000000,7454.000000,7454.000000,4377.000000,4377.000000,4377.000000,4377.000000,4377.000000,3969.000000,3969.000000,3969.000000,1496.000000,666.000000,3969.000000,3969.000000,3969.000000,3969.000000,3969.000000,3969.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2016.000000,3969.000000,3969.000000,3969.000000,3969.000000,3969.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259,NaN,241,68,115,NaN,NaN,NaN,269,409,261,NaN,NaN,NaN,367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [17]:
def nomalize_wave_preferences(row):
    # For all batch of preferences, rescale values to have sum() == 100
    for columns in [pref_1_1_cols, pref_4_1_cols, pref_2_1_cols, pref_1_2_cols]:
        preferences = row[columns]

        pref_total = preferences.sum()
        
        if pref_total > 0:
            # rescale pref_total -> 100 
            def normalize_pref(p):
                if p is None:
                    return None

                return ((p / pref_total) * 100)

            row[columns] = preferences.apply(normalize_pref)

    return row

waves_6_9_mask = speed_dating_df['wave'].between(6,9, inclusive='both')

speed_dating_df[waves_6_9_mask] = speed_dating_df[waves_6_9_mask].apply(nomalize_wave_preferences, axis=1)

print("Normalized:")
display(speed_dating_df.describe(include='all'))

Normalized:


,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
count,8368.000000,8367.000000,8368.00000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,6532.000000,8368.000000,8368.000000,8368.000000,8368.000000,8210.000000,8368.000000,8274.000000,8305.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,8368.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,8274.000000,8305,8286.000000,4914,3133,3583,8305.000000,8289.000000,8289.000000,8289,7305,4273,8289.000000,8271.000000,8289.000000,8279,8230.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8267.000000,1790.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,6489.000000,6489.000000,6489.000000,6489.000000,6489.000000,6467.000000,8289.000000,8289.000000,8289.000000,8289.000000,8280.000000,8280.000000,8263.000000,8263.000000,8263.000000,8263.000000,8263.000000,4906.000000,4906.000000,4906.000000,4906.000000,4906.000000,8368.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,7195.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,7454.000000,7454.000000,7424.000000,1984.000000,1955.000000,1984.000000,1984.000000,1955.000000,1974.000000,7436.000000,7454.000000,7454.000000,7454.000000,7454.000000,7454.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.00000,5775.000000,5775.000000,5775.000000,5775.000000,7454.000000,7454.000000,7454.000000,7454.000000,7454.000000,4377.000000,4377.000000,4377.000000,4377.000000,4377.000000,3969.000000,3969.000000,3969.000000,1496.000000,666.000000,3969.000000,3969.000000,3969.000000,3969.000000,3969.000000,3969.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2016.000000,3969.000000,3969.000000,3969.000000,3969.000000,3969.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259,NaN,241,68,115,NaN,NaN,NaN,269,409,261,NaN,NaN,NaN,367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Typage des données

In [18]:
# When numeric indexes, categorical columns can be converted to int
id_columns = ["iid", "id", "idg", "wave", "position", "positin1", "order", "partner", "pid"]
categorical_columns = ["gender", "condtn", "match", "dec", "dec_o", "samerace", "race", "race_o",
                       "field_cd", "goal", "go_out", "career_c", "date"]

speed_dating_df[id_columns] = speed_dating_df[id_columns].astype('Int64')
speed_dating_df[categorical_columns] = speed_dating_df[categorical_columns].astype('Int64')

speed_dating_df["gender_label"] = speed_dating_df["gender"].map(LabelDecoder.get_gender_label)

### Amélioration de la lisibilité du dataset
- Labels des variables catégorielles

In [19]:
# Gender
speed_dating_df["gender_label"] = speed_dating_df["gender"].map(LabelDecoder.get_gender_label)

# Race (race, race_o) Other = 6
speed_dating_df.fillna({"race": 6}, inplace=True)
speed_dating_df["race_label"] = speed_dating_df["race"].map(LabelDecoder.get_race_label)

speed_dating_df.fillna({"race_o": 6}, inplace=True)
speed_dating_df["race_label_o"] = speed_dating_df["race_o"].map(LabelDecoder.get_race_label)

# # Goal (goal) Other = 6
speed_dating_df.fillna({"goal": 6}, inplace=True)
speed_dating_df["goal_label"] = speed_dating_df["goal"].map(LabelDecoder.get_goal_label)

# # Field Coded (field_cd) Other = 18
speed_dating_df.fillna({"field_cd": 18}, inplace=True)
speed_dating_df["field_cd_label"] = speed_dating_df["field_cd"].map(LabelDecoder.get_field_cd_label)

# Career (career_c) Other = 15
speed_dating_df.fillna({"career_c": 15}, inplace=True)
speed_dating_df["career_c_label"] = speed_dating_df["career_c"].map(LabelDecoder.get_career_label)

In [20]:
display(speed_dating_df)

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,gender_label,race_label,race_label_o,goal_label,field_cd_label,career_c_label
0,1,1,0,1,1,1,10,7,<NA>,4,1,11,0,0.14,0,27.0,2,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1,NaN,NaN,NaN,4,2.0,4.0,Chicago,"60,521","69,487.00",2,7,1,lawyer,15,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN,Woman,Asian/Pacific Islander/Asian-American,European/Caucasian-American,To meet new people,Law,Other
1,1,1,0,1,1,1,10,7,<NA>,3,2,12,0,0.54,0,22.0,2,60.0,0.0,0.0,40.0,0.0,0.0,0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,Law,1,NaN,NaN,NaN,4,2.0,4.0,Chicago,"60,521","69,487.00",2,7,1,lawyer,15,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN,Woman,Asian/Pacific Islander/Asian-American,European/Caucasian-American,To meet new people,Law,Other
2,1,1,0,1,1,1,10,7,<NA>,10,3,13,1,0.16,1,22.0,4,19.0,18.0,19.0,18.0,14.0,12.0,1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1,NaN,NaN,NaN,4,2.0,4.0,Chicago,"60,521","69,487.00",2,7,1,lawyer,15,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.

Pour la suite, le code de chargment et de préparation du dataset est mutualisé dans le fichier [data_loader.py](data_loader.py)